In [2]:
from llama_cpp import Llama

In [23]:
llm = Llama(model_path=".\models\llama-2-7b-chat.Q4_K_S.gguf")
# output = llm("Q: Name the planets in the solar system? A: ", max_tokens=64, stop=["Q:", "\n"], echo=True)

1660856217200


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [1]:
import pandas as pd

df = pd.read_csv("main_data.csv")

df_html = df["html"][219:396]


In [2]:
df_html.to_csv('pandas.txt', header=None, index=None, sep=' ', mode='a')

In [72]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import CSVLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import LlamaCppEmbeddings
# split uusing token
from transformers import GPT2TokenizerFast
from langchain.text_splitter import CharacterTextSplitter

In [8]:
loader = TextLoader("pandas.txt")
documents = loader.load_and_split()
len(documents)

252

In [75]:
loader = CSVLoader(file_path="main_data.csv", csv_args={
                'delimiter': ','})
documents = loader.load()
len(documents)

474

In [82]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024, # changing this increased answer size ()
    chunk_overlap=20 
)
texts = text_splitter.split_documents(documents)
len(texts)

1409

In [73]:
# using a different tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=512, chunk_overlap=20
)
texts = text_splitter.split_text(documents)

TypeError: expected string or bytes-like object

In [83]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)
# embeddings = LlamaCppEmbeddings(model_path="./models/llama-2-7b-chat.Q4_K_S.gguf")

In [84]:
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

In [85]:
llm = LlamaCpp(
    model_path="./models/vicuna-13b-v1.5.Q4_K_M.gguf",
    n_ctx=2048,
    # verbose=True,
)

2617282214176


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [86]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True,
    verbose=False,
)

In [87]:
res = qa(f"""
    How to join CCI's new student server?
""")
print(res["result"])


    To join CCI's new student server, you can follow these steps:
    1. Go to the "Internal Servers" page for current students and click on the appropriate server.
    2. You will be prompted to log in with your Drexel username and password, as well as complete multi-factor authentication (MFA).
    3. Once you have successfully logged in, you should be able to join the new student server.


In [46]:
# Bring in deps
import streamlit as st
from langchain.llms import LlamaCpp
from langchain.embeddings import LlamaCppEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma


# Customize the layout
st.set_page_config(page_title="test", layout="wide")
st.markdown(f"""
            <style>
            .stApp {{background-image: url("https://images.unsplash.com/photo-1509537257950-20f875b03669?ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D&auto=format&fit=crop&w=1469&q=80");
                     background-attachment: fixed;
                     background-size: cover}}
         </style>
         """, unsafe_allow_html=True)

# function for writing uploaded file in temp
def write_text_file(content, file_path):
    try:
        with open(file_path, 'w') as file:
            file.write(content)
        return True
    except Exception as e:
        print(f"Error occurred while writing the file: {e}")
        return False

# set prompt template
# prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
# {context}
# Question: {question}
# Answer:"""
# prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# initialize hte LLM & Embeddings
llm = LlamaCpp(model_path="./models/llama-7b.ggmlv3.q4_0.bin")
embeddings = LlamaCppEmbeddings(model_path="models/llama-7b.ggmlv3.q4_0.bin")
# llm_chain = LLMChain(llm=llm, prompt=prompt)
llm_chain = LLMChain(llm=llm)

st.title("Document Conversation")
uploaded_file = st.file_uploader("Upload an article", type="txt")

if uploaded_file is not None:
    content = uploaded_file.read().decode('utf-8')
    # st.write(content)
    file_path = "temp/file.txt"
    write_text_file(content, file_path)

    loader = TextLoader(file_path)
    docs = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
    texts = text_splitter.split_documents(docs)
    db = Chroma.from_documents(texts, embeddings)
    st.success("File Loaded Successfully!!")

    # Query through LLM
    question = st.text_input("Ask something from the file", placeholder="Find something similar to: ....this.... in the text?", disabled=not uploaded_file,)
    if question:
        similar_doc = db.similarity_search(question, k=1)
        context = similar_doc[0].page_content
        # query_llm = LLMChain(llm=llm, prompt=prompt)
        query_llm = LLMChain(llm=llm)
        response = query_llm.run({"context": context, "question": question})
        st.write(response)

AttributeError: module 'streamlit' has no attribute '_is_running_with_streamlit'